In [1]:
from google.cloud import aiplatform

import tensorflow as tf
import numpy as np
import asyncio
import aiohttp
import json
import glob
import time
from PIL import Image
from statistics import mean
from io import BytesIO

In [24]:
# Setup endpoint

# CHANGE THESE
PROJECT_NUMBER = "827354046383"
ENDPOINT_ID = "4703099415160684544" #1node8core7.5gb
auth_token = "ya29.c.b0AUFJQsGD4I7d2uhUy6E4sd_XAQJuD5RDxOjUW59iRnh3TDMyNrITtlHJ7wOdDZVYRWAPTnCK7xQ18ITsmRu5B75461J5BOIU7vtwSh2eQ3PFqJg0QDTHSVnfnakj3w0tenN4Gx7lf7ymruGmYtut5zmHX0y5g4QKL5I-KJ4FI4TwM5sIUcJkcJiZiniex7E7UxlDQuoc4tr6hjnKRP78F62aV9mFP62ueURgiNUf"

endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/us-central1/endpoints/{ENDPOINT_ID}")

url="https://us-central1-aiplatform.googleapis.com/v1beta1/projects/{}/locations/us-central1/endpoints/{}:predict".format(PROJECT_NUMBER, ENDPOINT_ID)

# Download image labels
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [20]:
# Image prep, Set number of jobs in a batch

file = 'imagenet/*.jpg'
file_string = glob.glob(file)
IMAGE_SIZE = (200, 200)
instances = []
njobs = 10 # number of jobs per batch
offset = 100

for x in range(njobs):
    # print(file_string[x+offset])
    im = Image.open(file_string[x+offset])
    im = im.resize(IMAGE_SIZE)
    im = np.array(im)/255.0
    img_str = im.astype(np.float32).tolist()
    instances.append(img_str)

In [25]:
nbatches = 5 # number of batches

# Batch response times
batch_response_times = []
batch_response_times = [0 for i in range(nbatches)]

# interarrival times for batches
inter_arrival_times = []
inter_arrival_times = [3 for i in range(nbatches)]
inter_arrival_times[0] = 0

# arrival times of batches
arrival_times = []
arrival_times = [0 for i in range(nbatches)]
arrival_times[0]=round(inter_arrival_times[0],2)#arrival of first customer
#Generate arrival times
for i in range(1,nbatches):
    arrival_times[i]=round((arrival_times[i-1]+inter_arrival_times[i]),2)
    
print("Batch interarrival times {}".format(inter_arrival_times))
print("Batch arrival times      {}".format(arrival_times))

Batch interarrival times [0, 3, 3, 3, 3]
Batch arrival times      [0, 3, 6, 9, 12]


In [26]:
# Single prediction request
@asyncio.coroutine
async def predict_single(session: aiohttp.ClientSession, instances: list, curCust: int, offset: int, start_time: float) -> dict:
    job_sent_time = time.monotonic()
    headers = {"Authorization": "Bearer {}".format(auth_token)}
    payload = {"instances" : instances[curCust:(curCust+1)]}
    resp = await session.post(url, json=payload, headers=headers) 
    data = await resp.json()
    return data

# Batch prediction request
@asyncio.coroutine
async def predict_batch(session, instances, b, offset, start_time):
    # Simulating arrival times
    await asyncio.sleep(arrival_times[b])
    print("Starting batch {} at {}s after start".format(b, time.monotonic() - start_time))
    batch_start_time = time.monotonic()
    tasks = []
    for c in range(0,njobs):
        tasks.append(predict_single(session=session, instances=instances, curCust=c, offset=offset, start_time=start_time))
    jobs = await asyncio.gather(*tasks, return_exceptions=True)
    batch_finish_time = time.monotonic() - batch_start_time
    batch_response_times[b] = batch_finish_time
    print("Batch {} complete after {}s".format(b,batch_finish_time))
    
    return jobs

# Main event loop
async def main(instances):
    async with aiohttp.ClientSession() as session:
        start_time = time.monotonic()
        batch_tasks = []
        for b in range(0,nbatches):
            batch_tasks.append(predict_batch(session, instances, b, offset, start_time))
        batches = await asyncio.gather(*batch_tasks, return_exceptions=True)
        total_time = time.monotonic() - start_time
        print("{} batches of {} jobs each took {}s".format(nbatches, njobs, total_time))
        print("Response times {}".format(batch_response_times))
        print("Mean batch response time {}".format(mean(batch_response_times)))

        return batches

await main(instances)

Starting batch 0 at 0.00046148200090101454s after start
Batch 0 complete after 1.7266506260002643s
Starting batch 1 at 3.0032339000008506s after start
Batch 1 complete after 1.1337783760009188s
Starting batch 2 at 6.003542419999576s after start
Batch 2 complete after 1.2634183860009216s
Starting batch 3 at 9.002532533000704s after start
Batch 3 complete after 1.058412104999661s
Starting batch 4 at 12.00352174299951s after start
Batch 4 complete after 1.1114489920000779s
5 batches of 2 jobs each took 13.115254357000595s
Response times [1.7266506260002643, 1.1337783760009188, 1.2634183860009216, 1.058412104999661, 1.1114489920000779]
Mean batch response time 1.2587416970003686


[[{'predictions': [[1.27845867e-09,
     1.14391145e-08,
     3.91101516e-08,
     3.05184322e-09,
     1.54160489e-08,
     6.21256078e-08,
     1.23184449e-07,
     2.29856187e-08,
     1.09743524e-07,
     7.191026e-09,
     1.59415081e-09,
     1.61361913e-09,
     8.42383385e-10,
     1.37372309e-08,
     5.79430859e-09,
     3.79118292e-09,
     8.75150707e-09,
     2.98341107e-09,
     1.18413528e-08,
     2.82468e-09,
     5.68040504e-09,
     8.76329942e-09,
     3.40908035e-09,
     4.88315379e-08,
     3.20370752e-09,
     3.04610914e-09,
     1.47930046e-09,
     6.88137103e-09,
     1.11411791e-09,
     4.66781724e-09,
     3.0390408e-09,
     1.62719027e-09,
     3.57599861e-09,
     5.86608095e-08,
     1.9597147e-08,
     2.81484325e-09,
     1.39213681e-08,
     4.31294334e-09,
     4.2605417e-09,
     3.52003617e-08,
     3.59716412e-09,
     3.07241921e-09,
     2.34040254e-09,
     1.35069831e-08,
     1.71751169e-09,
     4.48841408e-09,
     8.80977158e-09,
     1